基本的にstacking.ipynbとほとんど同じだが、learn_modelの定義のみ異なる

In [6]:
#必要なものをインストール
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold

データセットをつくる
ホールドアウト法で大体の結果を確かめるためのテスト用

In [7]:
df_test=pd.read_csv("df_yy_all_log2.csv")#ここに入れるデータ名を入れ替えていろいろな例を試す
X = df_test.drop('y',axis=1).values
y=df_test.loc[:,["y"]].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
y_train= y_train.reshape(-1)
y_test= y_test.reshape(-1)

データセット　本番用


In [12]:
df=pd.read_csv("df_yy_all_log2.csv")
df_t=pd.read_csv('df_yy_all_log_test.csv')
X_train= df.drop('y',axis=1).values
y_train=df.loc[:,["y"]].values
X_test=df_t.values
y_train= y_train.reshape(-1)

学習モデルの作成

In [9]:
rfc=RandomForestClassifier(random_state=0, n_estimators=1000,max_depth= 15,
#                            class_weight="balanced"　:うまくy＝1が予測されるようにしようとするときに試した
                          )
gbct = GradientBoostingClassifier(n_estimators=5000,random_state=0, max_depth=15, learning_rate=0.01)
ada=AdaBoostClassifier(n_estimators=500, learning_rate=0.01, random_state=0)
ext=ExtraTreesClassifier(n_estimators=10000, max_depth =6)
rbf_svm = SVC(kernel='rbf', gamma=0.1, C=1,probability=True)
lgb = lgb.LGBMClassifier(num_leaves = 30,learning_rate=0.1,min_child_samples=20,n_estimators=2000,
#                          class_weight="balanced" :うまくy＝1が予測されるようにしようとするときに試した
                        )
xgb = xgb.XGBClassifier(learning_rate = 0.02,n_estimators= 2000,max_depth= 15,min_child_weight= 1, gamma=0.9,
#                         class_weight="balanced"
                       )

K個に分割してtrainデータの予測値を求める

オーバーサンプリング用

In [ ]:
def learn_model(model):
#yのクラスの比率が等しくなるようにホールドする
#     kf = KFold(n_splits=5, random_state=0, shuffle=True) :通常の分割方法
# 分けたときのyのバランスが良くなるようにしたもの    
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    ntrain=X_train.shape[0]
    ntest=X_test.shape[0]
#結果を記録していくためのものをつくる   
    mo_train=np.zeros((ntrain,))
    mo_test = np.zeros((ntest,))
    mo_test_m = np.empty((5, ntest))
    mo_train_pro=np.zeros((ntrain,))
    mo_test_pro = np.zeros((ntest,))
    mo_test_m_pro = np.empty((5, ntest))
    
    for i, (train_index, test_index) in enumerate(kf.split(X_train,y_train)):
    #分けたデータの1つは予測するためのテストデータとして扱い、残りのデータを使って学習する
        X_train1, X_test_1 = X_train[train_index], X_train[test_index]
        y_train1, y_test_1 = y_train[train_index], y_train[test_index]
        a=len(y_train1[y_train1==0])

        # ランダムにover-sampling
        ros = RandomOverSampler(ratio={0: a, 1: a//2}, random_state=0)
        X_train_1, y_train_1 = ros.fit_sample(X_train1, y_train1)

        model.fit(X_train_1,y_train_1)
        print('train accuracy: %.3f' % model.score(X_train_1, y_train_1))
        print('test accuracy: %.3f' % model.score(X_test_1, y_test_1))
        
        print(metrics.accuracy_score(y_test_1, model.predict(X_test_1)))
        # 再現率を出力
        print(metrics.recall_score(y_test_1, model.predict(X_test_1)))
        # 適合率を出力
        print(metrics.precision_score(y_test_1, model.predict(X_test_1)))
        # F値を出力
        print(metrics.f1_score(y_test_1, model.predict(X_test_1)))
        
        mo_train[test_index] =model.predict(X_test_1)
        mo_test_m[i, :] = model.predict(X_test)
        mo_train_pro[test_index] =model.predict_proba(X_test_1)[:,1]
        mo_test_m_pro[i, :] = model.predict_proba(X_test)[:,1]
    mo_test[:]=mo_test_m.mean(axis=0)
    mo_test_pro[:]=mo_test_m_pro.mean(axis=0)
   
    return mo_train.reshape(-1, 1), mo_test.reshape(-1, 1),mo_train_pro.reshape(-1, 1), mo_test_pro.reshape(-1, 1)

アンダーサンプリング用

In [ ]:
def learn_model(model):
#yのクラスの比率が等しくなるようにホールドする
#     kf = KFold(n_splits=5, random_state=0, shuffle=True) :通常の分割方法
# 分けたときのyのバランスが良くなるようにしたもの    
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    ntrain=X_train.shape[0]
    ntest=X_test.shape[0]
#結果を記録していくためのものをつくる   
    mo_train=np.zeros((ntrain,))
    mo_test = np.zeros((ntest,))
    mo_test_m = np.empty((5, ntest))
    mo_train_pro=np.zeros((ntrain,))
    mo_test_pro = np.zeros((ntest,))
    mo_test_m_pro = np.empty((5, ntest))
    
    for i, (train_index, test_index) in enumerate(kf.split(X_train,y_train)):
    #分けたデータの1つは予測するためのテストデータとして扱い、残りのデータを使って学習する
        X_train1, X_test_1 = X_train[train_index], X_train[test_index]
        y_train1, y_test_1 = y_train[train_index], y_train[test_index]
        # y＝1のサンプル数をカウント
        positive_count_train = y_train1.sum()
        print('positive count: {}'.format(positive_count_train))
        

        # ランダムにunder-sampling
        rus = RandomUnderSampler(ratio={0:positive_count_train*4, 1:positive_count_train}, random_state=0)
        X_train_1, y_train_1 = rus.fit_sample(X_train1, y_train1)

        model.fit(X_train_1,y_train_1)
        print('train accuracy: %.3f' % model.score(X_train_1, y_train_1))
        print('test accuracy: %.3f' % model.score(X_test_1, y_test_1))
        
        print(metrics.accuracy_score(y_test_1, model.predict(X_test_1)))
        # 再現率を出力
        print(metrics.recall_score(y_test_1, model.predict(X_test_1)))
        # 適合率を出力
        print(metrics.precision_score(y_test_1, model.predict(X_test_1)))
        # F値を出力
        print(metrics.f1_score(y_test_1, model.predict(X_test_1)))
        
        mo_train[test_index] =model.predict(X_test_1)
        mo_test_m[i, :] = model.predict(X_test)
        mo_train_pro[test_index] =model.predict_proba(X_test_1)[:,1]
        mo_test_m_pro[i, :] = model.predict_proba(X_test)[:,1]
    mo_test[:]=mo_test_m.mean(axis=0)
    mo_test_pro[:]=mo_test_m_pro.mean(axis=0)
    # print(mo_test.reshape(-1,1))
    return mo_train.reshape(-1, 1), mo_test.reshape(-1, 1),mo_train_pro.reshape(-1, 1), mo_test_pro.reshape(-1, 1)

学習させるモデルを順番に学習して、train,testの結果を受け取る

In [1]:
#Gridsearchにより最適なパラメータを求める（RandomForest）

# params = {'n_estimators'  : [ 10, 100, 1000, 10000] }
# kf_1 = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
# gs_rf = GridSearchCV(RandomForestClassifier(), params, cv = kf_1)
# gs_rf.fit(X_train, y_train)
# print(gs_rf.score(X_train,y_train))
# print(gs_rf.best_params_)

In [2]:
#RandomForestの学習
print("rfc")
rfc_train,rfc_test,rfc_train_pro,rfc_test_pro=learn_model(rfc)

In [3]:
#Gridsearch:GrasientBoost

# params = {'n_estimators'  : [ 5000,8000], 
#          'learning_rate' :[0.01,0.005]}
# gs_gb = GridSearchCV(GradientBoostingClassifier(), params, cv = kf_1)
# gs_gb.fit(X_train, y_train)
# print(gs_gb.score(X_train,y_train))

In [4]:
# print(gs_gb.best_params_)

In [5]:
#GrasientBoostの学習
print("gbct")
gbct_train,gbct_test,gbct_train_pro,gbct_test_pro=learn_model(gbct)

In [ ]:
#Gridsearch:

# params = {'n_estimators'  : [ 10, 100, 1000, 10000], 
#          'learning_rate' :[0.01,,1.0]}
# gs_ad = GridSearchCV(AdaBoostClassifier(), params, cv = kf_1)
# gs_ad.fit(X_train, y_train)
# print(gs_ad.score(X_train,y_train))

In [6]:
#AdaBoostの学習
print("ad")
ada_train, ada_test,ada_train_pro, ada_test_pro=learn_model(ada)


In [ ]:
#Grisserch: ExtraTreesClassifier

# params = {'n_estimators'  : [10, 100, 1000, 10000], 
#          }
# gs_ex = GridSearchCV(ExtraTreesClassifier(), params, cv = kf_1)
# gs_ex.fit(X_train, y_train)
# print(gs_ex.score(X_train,y_train))

In [7]:
#ExtraTreesClassifierの学習
print("ext")
ext_train, ext_test,ext_train_pro, ext_test_pro=learn_model(ext)

In [1]:
#GridSearch :SVC

# param_grid = {'C': [0.1, 1.0, 10],
#  'gamma': [0.001, 0.01]}
# gs_svc = GridSearchCV(SVC(), param_grid, cv=kf_1)
# gs_svc.fit(X_train,y_train)

In [ ]:
# gs_svc.best_params_

In [2]:
#SVCの学習（精度が悪く、学習時間が長いため使用しない）
# print("svc")
# svm_train, svm_test,svm_train_pro, svm_test_pro=learn_model(rbf_svm)

In [8]:
#lightGBMの学習
print("lgb")
lgb_train, lgb_test,lgb_train_pro, lgb_test_pro=learn_model(lgb)

In [9]:
#XGBoostの学習
print("xgb")
xgb_train, xgb_test,xgb_train_pro, xgb_test_pro=learn_model(xgb)

ロジスティック回帰追加

In [22]:
from sklearn.preprocessing import StandardScaler
#標準化
scaler = StandardScaler()
scaler.fit(X_train)
X_std_ba = scaler.transform(X_train)
X_std_test=scaler.transform(X_test)
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=1.0)


In [25]:
#引数にtrainデータとtestデータを指定する以外上のモデルと一緒（標準化処理を加えたから）
def learn_log(model,X_train,X_test):
    kf =StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    ntrain=X_train.shape[0]
    ntest=X_test.shape[0]
    mo_train=np.zeros((ntrain,))
    mo_test = np.zeros((ntest,))
    mo_test_m = np.empty((5, ntest))
    mo_train_pro=np.zeros((ntrain,))
    mo_test_pro = np.zeros((ntest,))
    mo_test_m_pro = np.empty((5, ntest))

    for i, (train_index, test_index) in enumerate(kf.split(X_train,y_train)):
        X_train_1, X_test_1 = X_train[train_index], X_train[test_index]
        y_train_1, y_test_1 = y_train[train_index], y_train[test_index]

        model.fit(X_train_1,y_train_1)
        print('train accuracy: %.3f' % model.score(X_train_1, y_train_1))
        print('test accuracy: %.3f' % model.score(X_test_1, y_test_1))
        # 再現率を出力
        print(metrics.recall_score(y_test_1, model.predict(X_test_1)))
        # 適合率を出力
        print(metrics.precision_score(y_test_1, model.predict(X_test_1)))
        # F値を出力
        print(metrics.f1_score(y_test_1, model.predict(X_test_1)))

        mo_train[test_index] =model.predict(X_test_1)
        mo_test_m_pro[i, :] = model.predict(X_test)
        mo_train_pro[test_index] =model.predict_proba(X_test_1)[:,1]
        mo_test_m_pro[i, :] = model.predict_proba(X_test)[:,1]
    mo_test[:]=mo_test_m.mean(axis=0)
    mo_test_pro[:]=mo_test_m_pro.mean(axis=0)
    # print(mo_test.reshape(-1,1))
    return mo_train.reshape(-1, 1), mo_test.reshape(-1, 1),mo_train_pro.reshape(-1, 1), mo_test_pro.reshape(-1, 1)

In [10]:
log_train, log_test, log_train_pro, log_test_pro=learn_log(lr,X_std_ba,X_std_test)

1回目の学習を行ったデータが消えてしまうとまた実行し直すのが大変だからcsvファイルにしておく

In [11]:
#trainデータの1回目の学習で予測したラベル0か1
#今回は基本的にラベルではなく確率を使用

# base_pre_train = pd.DataFrame( {'RandomForest': rfc_train.ravel(),
#      'ExtraTrees': ext_train.ravel(),
#      'AdaBoost': ada_train.ravel(),
# #         'svm': svm_train.ravel(),
#       'GradientBoost': gbct_train.ravel(),
#          "Logistic":log_train.ravel(),
#         "lightGBM":lgb_train.ravel(),
#         "XGBoost":xgb_train.ravel()
                            
                                
#     })
# print('base_pre_train.shape : ', base_pre_train.shape)
# base_pre_train.head(5)

In [12]:
#1回目の学習でtrainデータのラベルを予測した時の口座開設確率
base_pre_train_pro = pd.DataFrame( {'RandomForest': rfc_train_pro.ravel(),
     'ExtraTrees': ext_train_pro.ravel(),
     'AdaBoost': ada_train_pro.ravel(),
#         'svm': svm_train.ravel(),
      'GradientBoost': gbct_train_pro.ravel(),
         "Logistic":log_train_pro.ravel(),
        "lightGBM":lgb_train_pro.ravel(),
        "XGBoost":xgb_train_pro.ravel()
                            
                                
    })
print('base_pre_train_pro.shape : ', base_pre_train_pro.shape)
base_pre_train_pro.head(5)

In [13]:
#1回目の学習でtestデータのラベルを予測した時の口座開設確率
base_pre_test_pro = pd.DataFrame( {'RandomForest': rfc_test_pro.ravel(),
     'ExtraTrees': ext_test_pro.ravel(),
     'AdaBoost': ada_test_pro.ravel(),
#         'svm': svm_train_pro.ravel(),
      'GradientBoost': gbct_test_pro.ravel(),
      "Logistic":log_test_pro.ravel(),
        "lightGBM":lgb_test_pro.ravel(),
        "XGBoost":xgb_test_pro.ravel()
                                    
    })
print('base_pre_train.shape : ', base_pre_test_pro.shape)
base_pre_test_pro.head(5)

In [5]:
#testデータの1回目の学習で予測したラベル0か1

# base_pre_test = pd.DataFrame( {'RandomForest': rfc_test.ravel(),
#      'ExtraTrees': ext_test.ravel(),
#      'AdaBoost': ada_test.ravel(),
# #         'svm': svm_train_pro.ravel(),
#       'GradientBoost': gbct_test.ravel(),
#       "Logistic":log_test.ravel(),
#         "lightGBM":lgb_test.ravel(),
#         "XGBoost":xgb_test.ravel()
                                    
#     })
# print('base_pre_test.shape : ', base_pre_test.shape)
# base_pre_test.head(5)

以下で保存

In [ ]:
base_pre_train_pro.to_csv("base_pre_train_pro2.csv")

In [ ]:
base_pre_train.to_csv("base_pre_train_pro2.csv")

In [ ]:
base_pre_test.to_csv("base_pre_test2.csv")

In [ ]:
base_pre_test_pro.to_csv("base_pre_test_pro2.csv")

2回目の学習を行うためのデータの作成

In [14]:
x_train = np.concatenate((  rfc_train_pro,  gbct_train_pro,log_train_pro
#                           svc_train_pro
                          ,ada_train_pro,ext_train_pro,log_train_pro,lgb_train_pro,xgb_train_pro), axis=1)
x_test = np.concatenate((  rfc_test_pro,  gbct_test_pro, log_test_pro
#                           svc_test_pro
                         ,ada_test_pro,ext_test_pro,log_test_pro, lgb_test_pro,xgb_test_pro
                        ), axis=1)
print('x_train.shape : ', x_train.shape)
print('x_test.shape : ', x_test.shape)

In [ ]:
#こちらは０、1で予測したものを学習させるときのデータ

# x_train1 = np.concatenate((  rfc_train,  gbct_train,log_train
#                           svc_train,ada_train,ext_trainlog_train, lgb_train,xgb_train
#                          ), axis=1)
# x_test1 = np.concatenate((  rfc_test,  gbct_test,log_test 
#                           svc_test,ada_test,ext_test,log_test, lgb_test,xgb_test
#                         ), axis=1)
# print('x_train.shape : ', x_train1.shape)
# print('x_test.shape : ', x_test1.shape)

2回目の学習RandomForest

In [32]:
rfc_3=RandomForestClassifier(random_state=0, n_estimators=500)
rfc_3.fit(x_train, y_train)
predictions = rfc_3.predict(x_test)
predictions_pro = rfc_3.predict_proba(x_test)

In [ ]:
# rfc_3=RandomForestClassifier(random_state=0, n_estimators=500)
# rfc_3.fit(x_train1, y_train)
# predictions1 = rfc_3.predict(x_test1)

テスト用のみ


In [ ]:
rfc_3.score(x_test,y_test)

In [ ]:
rfc_3.score(x_train1,y_train)

ロジスティック

In [33]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)
X_std = scaler.transform(x_train)
x_test_std=scaler.transform(x_test)


In [15]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=1.0)
lr.fit(X_std, y_train)


テスト用のみ

In [ ]:
# 正答率を出力
print('train acc: %.3f' % lr.score(X_std, y_train))
print('test acc: %.3f' % lr.score(x_test_std, y_test))


XGBoost

In [35]:
import xgboost as xgb
xgb = xgb.XGBClassifier(learning_rate=0.1,max_depth=10)
xgb.fit(x_train,y_train)
pre_xgb = xgb.predict_proba(x_test)


テスト用のみ

In [ ]:
# 正答率を出力
print('train acc: %.3f' % xgb.score(x_train, y_train))
print('test acc: %.3f' % xgb.score(x_test, y_test))


csvファイルにして保存

In [ ]:
predic_df=pd.DataFrame(predictions_pro)
predic_df.to_csv("result_rf_new2_pro_log3.csv")
predic_log_pro=lr.predict_proba(x_test_std)
predic_log_df=pd.DataFrame(predic_log_pro)
pre_xgb=pd.DataFrame(pre_xgb)
predic_log_df.to_csv("result_log_new2_pro_log3.csv")
pre_xgb.to_csv("result_xgb_pro.csv")